In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
import socket, struct
from sklearn.preprocessing import OneHotEncoder, normalize, MinMaxScaler
from sklearn.model_selection import KFold
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
import time
import keras_metrics as km
from keras.layers import merge, Input, Dropout, Concatenate,Activation
from keras.models import Model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from keras.callbacks import Callback

class SkMetrics(Callback):
    def __init__(self,batchsize):
        self.batchsize = batchsize
    def on_train_begin(self, logs={}):
        self.confusion = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.kappa = []        

    def on_epoch_end(self, epoch, logs={}):
        score = numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize))
        predict = numpy.round(numpy.asarray(self.model.predict(self.validation_data[0],batch_size = self.batchsize)))
        targ = self.validation_data[1]
        #targ = targ.flatten()
        #predict = predict.flatten()
        self.confusion.append(multilabel_confusion_matrix(targ, predict))
        #self.precision.append(precision_score(targ, predict))
        #self.recall.append(recall_score(targ, predict))
        #self.f1s.append(f1_score(targ, predict, average="macro"))
        #self.kappa.append(cohen_kappa_score(targ, predict))

def getModel(neurons, layers, inputshape, batchsize, numAttacks):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape = (batchsize, inputshape[0], inputshape[1]), return_sequences=True, stateful=False))
    model.add(LSTM(neurons, return_sequences=True, stateful=False))
    model.add(LSTM(neurons, return_sequences=False, stateful=False))
    model.add(Dense(numAttacks, activation = 'sigmoid'))
    model.compile(optimizer="adam", 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./UNSW-NB15_1.csv', header=None)
data1a = numpy.asarray(data)

data2 = pd.read_csv('./UNSW-NB15_2.csv', header=None)
data2a = numpy.asarray(data2)

data3 = pd.read_csv('./UNSW-NB15_3.csv', header=None)
data3a = numpy.asarray(data3)

data = []
temp = len(data1a) + len(data2a) + len(data3a)
data.append(data1a)
data.append(data2a)
data.append(data3a)

data = numpy.reshape(numpy.asarray(data),(temp, 49))


/home/vanet/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/vanet/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def iptoint(ip):
    return struct.unpack("!I",(socket.inet_aton(ip)))[0] 

def parse(data):
    protocols = dict()
    protocolsNum = 1
    services  = dict()
    servicesNum = 1
    states = dict()
    statesNum = 1
    attacksTypes = dict()
    attacksNum = 1
    numMalicious = 0
    numNormal = 0
    x = []
    y = []
    numMaliciousTypes = []
    for dataRaw in data:
        dataRaw[0] = iptoint(dataRaw[0])
        dataRaw[2] = iptoint(dataRaw[2])
        #Source IP
        if (type(dataRaw[39]) == str):
            if (dataRaw[39] !=  " "):
                dataRaw[39] = int(dataRaw[39],0)
            else:
                dataRaw[39] = 0
        if (type(dataRaw[1]) == str):
            if (dataRaw[1] != "-"):
                dataRaw[1] = int(dataRaw[1],0)
            else:
                dataRaw[1] = 0
        #Dest IP
        if (type(dataRaw[3]) == str):
            if (dataRaw[3] != "-"):
                dataRaw[3] = int(dataRaw[3],0)
            else:
                dataRaw[3] = 0
        if (dataRaw[4] in protocols):
            dataRaw[4] = protocols.get(dataRaw[4])
        else:
            protocols.update({dataRaw[4]:protocolsNum})
            dataRaw[4] = protocolsNum
            protocolsNum += 1

        if (type(dataRaw[4])!= int):
            print (dataRaw[4])
 
        if (dataRaw[5] in states):
            dataRaw[5] = states.get(dataRaw[5])
        else:
            states.update({dataRaw[5]:statesNum})
            dataRaw[5] = statesNum
            statesNum += 1
        if (type(dataRaw[5])!=int):
            print (dataRaw[5])
        if (dataRaw[13] in services):
            dataRaw[13] = services.get(dataRaw[13])
        else:
            services.update({dataRaw[13]:servicesNum})
            dataRaw[13] = servicesNum
            servicesNum += 1
        if (type(dataRaw[13])!=int):
            print (dataRaw[13])
        if (dataRaw[48] == 1):
            numMalicious = numMalicious + 1
            if (dataRaw[47] in attacksTypes):
                y.append(attacksTypes.get(dataRaw[47]))
                numMaliciousTypes[attacksTypes.get(dataRaw[47])-1]+=1
            else:
                attacksTypes.update({dataRaw[47]:attacksNum})
                y.append(attacksNum)
                attacksNum+=1
                numMaliciousTypes.append(1)
        else:
            numNormal = numNormal + 1
            y.append(0)
        i = 0
        #for data in dataRaw:
        #    if (type(data) == str):
        #        print(dataRaw,data,i)
        #    i = i+1
        x.append(dataRaw)
    print("normal = ",numNormal)
    print("Malicious = ", numMalicious)
    print("List TypesMalicousNum",numMaliciousTypes)
    print("Attacks: ",attacksTypes)
    print(protocols)
    print(states)
    print(services)
    return [x, y, attacksNum]


def optimizeParameters(data, labels, scorefunc):
    # Выбираем k лучших признаков
    selecter = SelectKBest(score_func=scorefunc, k=10)
    selecter.fit(data, labels)
    
    # Какие именно выбрали
    params = []
    index = []
    params = selecter.get_support()
    count = 0
    
    for param in params :
        if param == True :
            index.append(count)
        count += 1
    

    index_file = open("index.txt" , "w")
    for i in index : 
        index_file.write(str(i) + " ")
    index_file.close()

    return selecter.transform(data)

def drawGraphic(dataset, labels, normalCount, malwareCount):

    parametersCount = len(dataset[0])

    normalCountList = [0] * parametersCount
    malwareCountList = [0] * parametersCount

    for i in range(0, len(dataset)):
        if(i < normalCount):
            for j in range(0, parametersCount):
                normalCountList[j] += dataset[i][j]
        else:
            for j in range(0, parametersCount):
                malwareCountList[j] += dataset[i][j]

    for i in range(0, parametersCount):
        normalCountList[i] = normalCountList[i]
        malwareCountList[i] = malwareCountList[i] 

    ts = pd.DataFrame({'normal': normalCountList,
                       'malware': malwareCountList})
    ts.plot()
    plt.show()
    return


    
def ROC(ROC_MODEL , ROC_X, ROC_Y):
    probs = ROC_MODEL.predict(ROC_X)
    ROC_Y = ROC_Y.flatten()
    probs = probs.flatten()
    FPr, TPr, threshold = metrics.roc_curve(ROC_Y, probs)
    roc_auc = metrics.auc(FPr, TPr)
    lw = 2
    
    plt.figure()
    plt.plot(FPr, TPr, lw = lw , label='Logistic Regression (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()

In [4]:

[x, y, attacksNum] = parse(data)
min_max_scaler = MinMaxScaler()
newx = numpy.delete(numpy.asarray(x),47,1)
newx = numpy.delete(numpy.asarray(newx),47,1)
newx = min_max_scaler.fit_transform(newx)

newy = y
print(len(newx))
newx = newx.tolist()
print(newx[0])
print(newy[0])

normal =  1867614
Malicious =  232389
List TypesMalicousNum [33086, 10457, 11446, 153603, 1140, 18856, 131, 1663, 2007]
Attacks:  {'Exploits': 1, 'Reconnaissance': 2, 'DoS': 3, 'Generic': 4, 'Shellcode': 5, 'Fuzzers': 6, 'Worms': 7, 'Backdoors': 8, 'Analysis': 9}
{'udp': 1, 'arp': 2, 'tcp': 3, 'ospf': 4, 'icmp': 5, 'igmp': 6, 'sctp': 7, 'udt': 8, 'sep': 9, 'sun-nd': 10, 'swipe': 11, 'mobile': 12, 'pim': 13, 'rtp': 14, 'ipnip': 15, 'ip': 16, 'ggp': 17, 'st2': 18, 'egp': 19, 'cbt': 20, 'emcon': 21, 'nvp': 22, 'igp': 23, 'xnet': 24, 'argus': 25, 'bbn-rcc': 26, 'chaos': 27, 'pup': 28, 'hmp': 29, 'mux': 30, 'dcn': 31, 'prm': 32, 'trunk-1': 33, 'xns-idp': 34, 'trunk-2': 35, 'leaf-1': 36, 'leaf-2': 37, 'irtp': 38, 'rdp': 39, 'iso-tp4': 40, 'netblt': 41, 'mfe-nsp': 42, 'merit-inp': 43, '3pc': 44, 'xtp': 45, 'idpr': 46, 'tp++': 47, 'ddp': 48, 'idpr-cmtp': 49, 'ipv6': 50, 'il': 51, 'idrp': 52, 'ipv6-frag': 53, 'sdrp': 54, 'ipv6-route': 55, 'gre': 56, 'rsvp': 57, 'mhrp': 58, 'bna': 59, 'esp': 60,

In [ ]:
from keras.utils import np_utils
while(len(newx)%100000 != 0):
    newx.pop()
    newy.pop()
newy = np_utils.to_categorical(newy)
print(attacksNum)
#enc = OneHotEncoder()
#newy = enc.fit_transform(y)
print(newy[0])
print(newy[1])
print(newy[2])
print(newy[3])
print(len(newx))

10
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
2100000


In [ ]:
print (newx[0])
print (len(newx[0]))
print (len(newx))

def construct_data_for_package_memory_len(package_len, x, y):
    x_result = []
    y_result = []
    for j in range (len(x) // package_len - 1):
        for i in range (package_len):
            temp_arr = []
            for k in range (package_len):
                temp_arr.append(x[j * package_len + i + k])
            x_result.append(temp_arr)
            y_result.append(y[j * package_len + i + package_len - 1])
    while len(x_result) % 10000 != 0:
        x_result.pop()
        y_result.pop()
    #x_result = numpy.asarray(x_result)
    #y_result = numpy.asarray(y_result)
    for i in range(len(x_result)):
        for j in range(len(x_result[i])):
            for k in range(len(x_result[i][j])):
                if numpy.isnan(x_result[i][j][k]):
                    x_result[i][j][k] = 1
    return [x_result, y_result]


[0.2711794028719652, 0.021210040436408027, 0.6524071462129994, 9.83321850267745e-08, 0.0, 0.0, 1.2006868117486436e-07, 9.194906523326434e-06, 1.1188787524993125e-05, 0.12156862745098039, 0.11417322834645668, 0.0, 0.0, 0.0, 8.357947830722337e-05, 0.0048290753567918655, 0.00018786398647379298, 0.0001815211472136504, 0.0, 0.0, 0.0, 0.0, 0.043882978723404256, 0.05466666666666666, 0.0, 0.0, 0.0, 0.0, 1.593020328982675e-05, 0.0, 2.8328649663255683e-07, 2.185413140586619e-07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030303030303030304, 0.09090909090909091, 0.0, 0.030303030303030304, 0.0, 0.0, 0.0]
47
2100000


In [ ]:
from sklearn.metrics import classification_report
def CrossValidation(lenOfBlock , x, y, neurons, layers, inputshape, epochsIn, batchsize, attacksNum):
    cv = KFold(n_splits = lenOfBlock, random_state = None, shuffle = True)
        
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    F1 = 0
    labels1 = []
    for i in range(attacksNum):
        labels1.append(i)
    labels1 = np_utils.to_categorical(labels1)
    #print(labels1)
    cn = multilabel_confusion_matrix(labels1, labels1)
    rescn = cn
    for j in range (len(cn)):
        for g in range (len(cn[0])):
            for d in range(len(cn[0][0])):
                rescn[j][g][d]=0
    class_weights = {0 : 1.,
                     1 : 55.,
                     2 : 172.,
                     3 : 157.,
                     4 : 12.,
                     5 : 1580.,
                     6 : 95.,
                     7 : 13000.,
                     8 : 1080.,
                     9 : 895.}
    Educate = 0.0
    Test = 0.0
    print("Neurons: ", neurons)
    print("Layers: ", layers)
    numblock=0
    x = numpy.reshape(x,(5,len(x)//5,inputshape[0],47))
    y = numpy.reshape(y,(5,len(y)//5,len(y[0])))
    for train_index, test_index in cv.split(x[0]):
        clf = getModel(neurons, layers, inputshape, batchsize, attacksNum)
        print("TRAIN:", train_index, "TEST:", test_index)
        print("Block: ", numblock)
        numblock +=1
        start_time = time.time()
        skmetrics = SkMetrics(batchsize)
        for i in range (epochsIn):
            history = clf.fit(x[0][train_index], y[0][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[0][test_index], y[0][test_index]), batch_size = batchsize, class_weight=class_weights)
            history = clf.fit(x[1][train_index], y[1][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[1][test_index], y[1][test_index]), batch_size = batchsize, class_weight=class_weights)
            history = clf.fit(x[2][train_index], y[2][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[2][test_index], y[2][test_index]), batch_size = batchsize, class_weight=class_weights)
            history = clf.fit(x[3][train_index], y[3][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[3][test_index], y[3][test_index]), batch_size = batchsize, class_weight=class_weights)
            history = clf.fit(x[4][train_index], y[4][train_index], callbacks = [skmetrics], epochs = 1, validation_data = (x[4][test_index], y[4][test_index]), batch_size = batchsize, class_weight=class_weights)
            i = i
        #for k in skmetrics.confusion:
        #    print(k)
        education_time = time.time() - start_time
        try:
            # Тестирование
            start_time = time.time()
            proba = []
            for i in range(5):
                proba.append(numpy.round(numpy.asarray(clf.predict(x[i][test_index], batch_size = batchsize))))

            test_time = time.time() - start_time
            # Добавляем значения
            Educate += education_time
            Test += test_time
            y1 = []
            for i in range (5):
                y1.append(y[i][test_index])
            cn = []
            print("CN: ")
            for i in range(5):
                cn.append(multilabel_confusion_matrix(y1[i], proba[i]))
                print(classification_report(y1[i], proba[i]))
                #print(cn[i])
                #tn1, fp1, fn1, tp1 = cn[i].ravel()
            for mat in range(len(cn)):
                for j in range (len(cn[0])):
                    for g in range (len(cn[0][0])):
                        for d in range(len(cn[0][0][0])):
                            rescn[j][g][d]+=cn[mat][j][g][d]
            print(rescn)
            #tn1, fp1, fn1, tp1 = cn1.ravel()
        except ZeroDivisionError:
            print("ZeroDivisionError")
        #TN += tn1 
        #FP += fp1 
        #FN += fn1
        #TP += tp1 
        #F1 += (f1_score(y1, proba , average='macro'))
        
    #summ = TN + FP + FN + TP
    
    print("\n\n\n")
    print("Block len " + str(lenOfBlock))
    #print("True positive " + str(TP / summ), TP)
    #print("True negative " + str(TN / summ), TN)
    #print("False positive " + str(FP / summ), FP)
    #print("False negative " + str(FN / summ), FN)
   
    #print("Accuracy " + str((TP + TN)/(summ)))
    #print("Precision " + str(TP / (TP + FP))) # Precision
    #print("Recall " + str(TP / (TP + FN))) # Recall

    #print("F1 score " + str(F1 / lenOfBlock)) # F1 score
    print("Educate time " + str(Educate / lenOfBlock)) # Educate time
    print("Test time " + str(Test / lenOfBlock)) # Test time

    print("\n\n\n")    

In [ ]:
#[x, y] = construct_data_for_package_memory_len(5, newx, newy)
#x1 = numpy.asarray(x)
#y1 = numpy.asarray(y)
#CrossValidation(5 , x1, y1, 200, 1, (5,47), 2, 200, attacksNum)


In [ ]:
#[x, y] = construct_data_for_package_memory_len(10, newx, newy)
#x1 = numpy.asarray(x)
#y1 = numpy.asarray(y)
#CrossValidation(5 , x1, y1, 200, 1, (10,47), 2, 200, attacksNum)


In [ ]:

[x, y] = construct_data_for_package_memory_len(5, newx, newy)
x1 = numpy.asarray(x)
y1 = numpy.asarray(y)
CrossValidation(5 , x1, y1, 200, 1, (5,47), 3, 400, attacksNum)



#[x, y] = construct_data_for_package_memory_len(20, newx, newy)
#x1 = numpy.asarray(x)
#y1 = numpy.asarray(y)
#CrossValidation(5 , x1, y1, 200, 1, (20,47), 2, 200)


Neurons:  200
Layers:  1
TRAIN: [     0      1      2 ... 417997 417998 417999] TEST: [     4     13     14 ... 417985 417990 417991]
Block:  0
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 193s 578us/step - loss: 16.0396 - acc: 0.6595 - val_loss: 1.2680 - val_acc: 0.8956
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 186s 555us/step - loss: 0.0393 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 190s 569us/step - loss: 16.6826 - acc: 0.9279 - val_loss: 0.8732 - val_acc: 0.9283
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [==============================] - 186s 557us/step - loss: 31.8637 - acc: 0.7551 - val_loss: 2.3917 - val_acc: 0.1243
Train on 334400 samples, validate on 83600 samples
Epoch 1/1
334400/334400 [=========

/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      0.86      0.92     79023
           1       0.83      0.11      0.19      1078
           2       0.53      0.35      0.42       380
           3       0.14      0.39      0.20       230
           4       1.00      0.00      0.00      1526
           5       0.08      0.50      0.14        58
           6       0.32      0.45      0.37      1087
           7       0.01      0.17      0.02         6
           8       0.14      0.86      0.23       101
           9       0.17      0.81      0.28       111

   micro avg       0.96      0.82      0.89     83600
   macro avg       0.42      0.45      0.28     83600
weighted avg       0.98      0.82      0.88     83600
 samples avg       0.82      0.82      0.82     83600



/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/vanet/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      0.92      0.96     83600
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

   micro avg       1.00      0.92      0.96     83600
   macro avg       0.10      0.09      0.10     83600
weighted avg       1.00      0.92      0.96     83600
 samples avg       0.92      0.92      0.92     83600

              precision    recall  f1-score   support

           0       1.00      0.91      0.95     77608
           1       0.72      0.19      0.30      1025
           2       0.76 

              precision    recall  f1-score   support

           0       1.00      0.93      0.96     77610
           1       0.55      0.05      0.10      1037
           2       0.51      0.51      0.51       358
           3       0.36      0.79      0.49       497
           4       1.00      0.58      0.74      3515
           5       0.05      0.76      0.10        29
           6       0.44      0.25      0.32       410
           7       0.01      0.60      0.03         5
           8       0.05      0.87      0.10        67
           9       0.05      0.75      0.10        72

   micro avg       0.95      0.90      0.92     83600
   macro avg       0.40      0.61      0.34     83600
weighted avg       0.98      0.90      0.93     83600
 samples avg       0.89      0.90      0.89     83600

              precision    recall  f1-score   support

           0       1.00      0.87      0.93     65997
           1       0.60      0.04      0.08      2678
           2       0.46 

              precision    recall  f1-score   support

           0       1.00      0.94      0.97     66431
           1       0.86      0.05      0.10      2578
           2       0.41      0.42      0.41       723
           3       0.27      0.73      0.40       854
           4       1.00      0.95      0.97     11562
           5       0.04      0.83      0.07        76
           6       0.40      0.30      0.34      1181
           7       0.01      0.25      0.02         4
           8       0.03      0.73      0.05        83
           9       0.04      0.69      0.08       108

   micro avg       0.90      0.89      0.90     83600
   macro avg       0.41      0.59      0.34     83600
weighted avg       0.97      0.89      0.92     83600
 samples avg       0.89      0.89      0.89     83600

              precision    recall  f1-score   support

           0       1.00      0.94      0.97     65064
           1       0.91      0.07      0.13      1838
           2       0.39 

              precision    recall  f1-score   support

           0       1.00      0.91      0.95     65202
           1       0.64      0.11      0.19      1872
           2       0.40      0.43      0.41       657
           3       0.34      0.68      0.46       679
           4       1.00      0.97      0.99     13817
           5       0.05      0.84      0.09        82
           6       0.33      0.62      0.43      1089
           7       0.01      0.80      0.01         5
           8       0.05      0.68      0.09        95
           9       0.04      0.49      0.07       102

   micro avg       0.91      0.89      0.90     83600
   macro avg       0.39      0.65      0.37     83600
weighted avg       0.97      0.89      0.92     83600
 samples avg       0.88      0.89      0.89     83600

[[[ 185380       2]
  [ 101423 1385195]]

 [[1644482    1125]
  [  23686    2707]]

 [[1659129    4555]
  [   4842    3474]]

 [[1647569   15319]
  [   2564    6548]]

 [[1549058     188]

In [ ]:


[x, y] = construct_data_for_package_memory_len(3, newx, newy)
x1 = numpy.asarray(x)
y1 = numpy.asarray(y)
CrossValidation(5 , x1, y1, 200, 1, (3,47), 2, 200)

In [ ]:
[x, y] = construct_data_for_package_memory_len(7, newx, newy)
x1 = numpy.asarray(x)
y1 = numpy.asarray(y)
CrossValidation(5 , x1, y1, 200, 1, (7,47), 2, 200)
